# 01.function

In [ ]:
# cytoTRACE
run_cytotrace <- function(sce,celltye){
    library(CytoTRACE)
    data <- as.matrix(sce@assays$RNA@counts)
    Component <- Embeddings(sce, reduction = "umap")
    cluster <- FetchData(object = sce, vars = c(celltye))
    clus1 <- as.character(cluster[,1])
    names(clus1) <- rownames(cluster)
    results <- CytoTRACE(data)

    results_cell <- as.data.frame(cbind(results[["CytoTRACE"]],results[["CytoTRACErank"]],
                                    results[["GCS"]],results[["Counts"]]))
    colnames(results_cell) <- c("CytoTRACE","CytoTRACErank","GCS","Counts")
    results_cell <- cbind(rownames(results_cell),results_cell)
    colnames(results_cell)[1] <- c("Barcode")
    cyto <- results_cell
    sce$Barcode<- rownames(sce[[]])
    meta <- FetchData(object = sce, vars = 'Barcode')
    meta <- left_join(meta, cyto,  by = 'Barcode')
    sce <- AddMetaData(object = sce,metadata = meta$CytoTRACE,col.name = c('CytoTRACE'))
    sce <- AddMetaData(object = sce,metadata = meta$CytoTRACErank,col.name = c('CytoTRACErank'))
    sce <- AddMetaData(object = sce,metadata = meta$GCS,col.name = c('GCS'))
    sce <- AddMetaData(object = sce,metadata = meta$Counts,col.name = c('Counts'))
}


#monocle2
run_monocle <- function(sce,markers){
    library(monocle)
    sce@meta.data$Barcode=colnames(sce)
    data <- as(as.matrix(sce@assays$RNA@counts), 'sparseMatrix')
    pd <- new('AnnotatedDataFrame', data = sce@meta.data)
    fData <- data.frame(gene_short_name = row.names(data), row.names = row.names(data))
    fd <- new('AnnotatedDataFrame', data = fData)
    cds <- newCellDataSet(data,
                          phenoData = pd,
                          featureData = fd,
                          lowerDetectionLimit = 0.5,
                          expressionFamily = negbinomial.size())

    cds <- estimateSizeFactors(cds)
    cds <- estimateDispersions(cds)
    cds <- detectGenes(cds, min_expr = 3 )
    ordering_genes <- (subset(markers, p_val_adj < 0.01))$gene
    length(ordering_genes)

    disp_table <- dispersionTable(cds)
    unsup_clustering_genes <- subset(disp_table, mean_expression >=0.1)
    common_strings <- intersect(unsup_clustering_genes$gene_id, ordering_genes)

    cds <- setOrderingFilter(cds, common_strings)
    plot_ordering_genes(cds)

    cds <- reduceDimension(cds, max_components = 3,
                           num_dim = 50,
                           method = 'DDRTree') # DDRTree方式
    #pData(cds)$State = pData(cds)$seurat_clusters

    #library(monocle)
    source("order_cells.R")
    cds <- orderCells(cds)
    meta <- FetchData(object = sce, vars = 'Barcode')
    pseudotime=cds@phenoData@data
    meta <- left_join(meta, pseudotime,  by = 'Barcode')

    plotdf2=as.data.frame(t(cds@reducedDimS))
    colnames(plotdf2)=c("component1","component2")

    sce <- AddMetaData(object = sce,metadata = meta$Pseudotime,col.name = c('Pseudotime'))
    sce <- AddMetaData(object = sce,metadata = meta$State,col.name = c('State'))
    sce <- AddMetaData(object = sce,metadata = plotdf2$component1,col.name = c('component1'))
    sce <- AddMetaData(object = sce,metadata = plotdf2$component2,col.name = c('component2'))
    return(list(mono = cds, rds = sce))
}
                                                    

# 02.standard analysis

In [ ]:
args <- commandArgs(TRUE)
rdsfile <- args[1]     # Expression data of oocytes merged with the 5-10 nearest spots to the oocyte center, expression data of granulosa cells randomly merged around oocytes at the same stage, and expression data of TC cells randomly merged around oocytes at the same stage
output_dir <- args[2]  # Output file path
prefix <- args[3]      # Prefix for output file

In [ ]:
micro <- readRDS(rdsfile)

In [10]:
Idents(micro) <- micro@meta.data$orig.ident

In [11]:
oo <- subset(micro,idents=c("OO"))
gc <- subset(micro,idents=c("GC"))
tc <- subset(micro,idents=c("TC"))

In [ ]:
Idents(oo)=oo@meta.data$subtype
markers <- FindAllMarkers(oo, only.pos =FALSE, min.pct = 0.1)
oo <- run_cytotrace(oo,"subtype")
Idents(oo)=oo@meta.data$subtype
result <- run_monocle(oo,markers)
cds <- result$mono
oo <- result$rds
saveRDS(cds,file=paste0(output_dir,"OO",".monocle.cds.rds"))
saveRDS(oo,file=paste0(output_dir,"OO",".monocle+cytotrace.rds"))

In [ ]:
Idents(gc)=gc@meta.data$subtype
markers <- FindAllMarkers(gc, only.pos =FALSE, min.pct = 0.1)
gc <- run_cytotrace(gc,"subtype")
Idents(gc)=gc@meta.data$subtype
result <- run_monocle(gc,markers)
cds <- result$mono
gc <- result$rds
saveRDS(cds,file=paste0(output_dir,"GC",".monocle.cds.rds"))
saveRDS(gc,file=paste0(output_dir,"GC",".monocle+cytotrace.rds"))

In [ ]:
Idents(tc)=tc@meta.data$subtype
markers <- FindAllMarkers(tc, only.pos =FALSE, min.pct = 0.1)
tc <- run_cytotrace(tc,"subtype")
Idents(tc)=tc@meta.data$subtype
result <- run_monocle(tc,markers)
cds <- result$mono
tc <- result$rds
saveRDS(cds,file=paste0(output_dir,"GC",".monocle.cds.rds"))
saveRDS(tc,file=paste0(output_dir,"GC",".monocle+cytotrace.rds"))
